In [11]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.utils import to_categorical

In [12]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [13]:
# Load the CSV file with paths and labels
csv_path = "fatiguelevel_updated.csv"
df = pd.read_csv(csv_path)

# Define the path to the normalized data folder
# norm_data_folder = "data_balanced"

# Initialize empty lists for features and labels
X = []
y = []


In [14]:
# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    file_path = os.path.join(row['file_loc'])
    
    # Load the normalized data from the CSV file
    data = pd.read_csv(file_path)
    
    # Extract the features (time series data) and label
    features = data.values[:, :]  # Exclude the last column (label)
    label = row['Stressed']
    
    # Add the features and label to the lists
    X.append(features)
    y.append(label)


In [15]:
X

[array([[0.53771252, 0.04657092, 0.25905782, ..., 0.13826921, 0.        ,
         0.18016391],
        [0.53741055, 0.11102678, 0.30812991, ..., 0.12510192, 0.        ,
         0.22567783],
        [0.53728555, 0.21201152, 0.31518558, ..., 0.13900896, 0.        ,
         0.242539  ],
        ...,
        [0.54905513, 0.27683326, 0.60914347, ..., 0.40284222, 0.21103604,
         0.5670182 ],
        [0.55120787, 0.29187296, 0.64734475, ..., 0.43926675, 0.26447072,
         0.55645075],
        [0.55294175, 0.30471742, 0.68432548, ..., 0.48396972, 0.33136261,
         0.54544433]]),
 array([[0.59781804, 0.40964503, 0.4159743 , ..., 0.15298632, 0.        ,
         0.81671306],
        [0.5986655 , 0.41240075, 0.4209636 , ..., 0.14873167, 0.        ,
         0.82375803],
        [0.59924995, 0.41583372, 0.42993576, ..., 0.1432028 , 0.        ,
         0.8311242 ],
        ...,
        [0.59661991, 0.47232602, 0.45192719, ..., 0.16381363, 0.        ,
         0.93037473],
        [0.5

In [16]:
# Convert the lists to NumPy arrays
X = np.array(X)
y = np.array(y)

In [17]:

y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
# Reshape the input data for CNN (assuming 1D convolution)
# The shape should be (num_samples, num_timesteps, num_features)
# If your data has a different shape, adjust accordingly
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

# Encode the labels (0 and 1) to categorical format
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

In [19]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Construct the CNN model
model = Sequential()
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
12/12 [==============================] - 2s 70ms/step - loss: 0.7777 - accuracy: 0.7711 - val_loss: 0.5712 - val_accuracy: 0.7609
Epoch 2/10
12/12 [==============================] - 0s 37ms/step - loss: 0.4738 - accuracy: 0.8147 - val_loss: 0.5675 - val_accuracy: 0.7609
Epoch 3/10
12/12 [==============================] - 0s 39ms/step - loss: 0.4521 - accuracy: 0.8147 - val_loss: 0.6093 - val_accuracy: 0.7609
Epoch 4/10
12/12 [==============================] - 0s 37ms/step - loss: 0.4382 - accuracy: 0.8147 - val_loss: 0.5837 - val_accuracy: 0.7609
Epoch 5/10
12/12 [==============================] - 0s 36ms/step - loss: 0.4559 - accuracy: 0.8147 - val_loss: 0.5603 - val_accuracy: 0.7609
Epoch 6/10
12/12 [==============================] - 0s 38ms/step - loss: 0.4259 - accuracy: 0.8147 - val_loss: 0.5924 - val_accuracy: 0.7609
Epoch 7/10
12/12 [==============================] - 1s 44ms/step - loss: 0.4087 - accuracy: 0.8147 - val_loss: 0.6432 - val_accuracy: 0.7609
Epoch 8/10
12

In [20]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:f}')
print(f'Test Accuracy: {accuracy:f}')

# Make predictions
predictions = model.predict(X_test)

3/3 [==============================] - 0s 8ms/step - loss: 0.5750 - accuracy: 0.7609
Test Loss: 0.575030
Test Accuracy: 0.760870
3/3 [==============================] - 0s 5ms/step
